In [50]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/oscarmontero/Google Drive/Ironhack_Data_Analytics_Bootcamp/ironhack_service_account_big_query.json"

from google.cloud import bigquery

client = bigquery.Client()

import six
six.moves.reload_module(six)

<module 'six' from '/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/six.py'>

# Challenge 1

## Step 1 - Calculate the royalties of each sales for each author

In [51]:
query_1 = '''

SELECT 
 authors.au_id AS AUTHOR_ID, 
 titles.title_id AS TITLE_ID, 
 titles.price * sales.qty * (titles.royalty / 100) * (titleauthor.royaltyper / 100) AS SALES_ROYALTY

FROM 
`ironhack-data-analytics-265219.publications.authors` authors

JOIN

`ironhack-data-analytics-265219.publications.titleauthor` titleauthor

ON

authors.au_id = titleauthor.au_id

JOIN

`ironhack-data-analytics-265219.publications.titles` titles

ON

titleauthor.title_id = titles.title_id

JOIN

`ironhack-data-analytics-265219.publications.sales` sales

ON

titles.title_id = sales.title_id

GROUP BY

AUTHOR_ID, 
TITLE_ID,
SALES_ROYALTY

ORDER BY
AUTHOR_ID
 
'''

In [52]:
query_job = client.query(query_1)
df = query_job.to_dataframe()
df

,AUTHOR_ID,TITLE_ID,SALES_ROYALTY
0,172-32-1176,PS3333,29.985
1,213-46-8915,BU1032,3.998
2,213-46-8915,BU1032,7.996
3,213-46-8915,BU2075,25.116
4,238-95-7766,PC1035,110.160
5,267-41-2394,BU1111,11.950
6,267-41-2394,TC7777,8.994
7,274-80-9391,BU7832,29.985
8,409-56-7008,BU1032,5.997
9,409-56-7008,BU1032,11.994


## Step 2: Aggregate the total royalties for each title for each author

In [53]:
query_2 = '''

SELECT 
 authors.au_id AS AUTHOR_ID, 
 titles.title_id AS TITLE_ID, 
 SUM(titles.price * sales.qty * (titles.royalty / 100) * (titleauthor.royaltyper / 100)) AS TOTAL_SALES_ROYALTY

FROM 
`ironhack-data-analytics-265219.publications.authors` authors

JOIN

`ironhack-data-analytics-265219.publications.titleauthor` titleauthor

ON

authors.au_id = titleauthor.au_id

JOIN

`ironhack-data-analytics-265219.publications.titles` titles

ON

titleauthor.title_id = titles.title_id

JOIN

`ironhack-data-analytics-265219.publications.sales` sales

ON

titles.title_id = sales.title_id

GROUP BY

AUTHOR_ID, 
TITLE_ID

ORDER BY
AUTHOR_ID
 
'''

In [54]:
query_job = client.query(query_2)
df = query_job.to_dataframe()
df

,AUTHOR_ID,TITLE_ID,TOTAL_SALES_ROYALTY
0,172-32-1176,PS3333,29.985
1,213-46-8915,BU1032,11.994
2,213-46-8915,BU2075,25.116
3,238-95-7766,PC1035,110.160
4,267-41-2394,BU1111,11.950
5,267-41-2394,TC7777,8.994
6,274-80-9391,BU7832,29.985
7,409-56-7008,BU1032,17.991
8,427-17-2319,PC8888,50.000
9,472-27-2349,TC7777,8.994


## Step 3: Calculate the total profits of each author

In [48]:
query_3 = '''

SELECT 
 authors.au_id AS AUTHOR_ID, 
 titles.title_id AS TITLE_ID, 
 SUM( (titles.price * sales.qty * (titles.royalty / 100) * (titleauthor.royaltyper / 100)) + titles.advance) AS TOTAL_PROFIT

FROM 
`ironhack-data-analytics-265219.publications.authors` authors

JOIN

`ironhack-data-analytics-265219.publications.titleauthor` titleauthor

ON

authors.au_id = titleauthor.au_id

JOIN

`ironhack-data-analytics-265219.publications.titles` titles

ON

titleauthor.title_id = titles.title_id

JOIN

`ironhack-data-analytics-265219.publications.sales` sales

ON

titles.title_id = sales.title_id

GROUP BY

AUTHOR_ID, 
TITLE_ID

ORDER BY
TOTAL_PROFIT DESC

LIMIT
3

'''

In [49]:
query_job = client.query(query_3)
df = query_job.to_dataframe()
df

,AUTHOR_ID,TITLE_ID,TOTAL_PROFIT
0,722-51-5454,MC3021,30021.528
1,899-46-2035,MC3021,30007.176
2,213-46-8915,BU2075,10150.116


# Challenge 2

In [ ]:
most_profit_authors = '''

CREATE TABLE 

publications.most_profiting_authors (

au_id STRING,
profits NUMERIC)

